In [ ]:
import pandas as pd
from pathlib import Path

# Set data paths

In [ ]:
# Explore data sources from .xslsx files, merge them and save them as .csv

brg_path = Path("data/2_brg/brg_raw.csv")
pkeg_path = Path("data/3_pkeg/pkeg_raw.csv")
old_brg_path = Path("data/4_brg_old/brg_old.csv")

DATA_COLS = ["source", "id", "lon", "lat", "date", "gwl_cm"]

In [ ]:
brg_df = pd.read_csv(brg_path)

# 1. Read data

### BRGM new data

In [ ]:
brg_cols_rename = {
    "gwl_rata" : "gwl_cm",
}

# rename columns
brg_df.rename(columns=brg_cols_rename, inplace=True)

# Convert date column to datetime
brg_df["date"] = pd.to_datetime(brg_df["date"])

# multiply gwl_cm by 100 to convert it to cm
brg_df["gwl_cm"] = brg_df["gwl_cm"] * 100

# set a new column for source
brg_df["source"] = "brg"

brg_df = brg_df[DATA_COLS]
brg_df.head()

In [ ]:
brg_df.id.unique()

### PKEG data

In [ ]:
pkeg_cols_rename = {
    "date(dd/mm/yyyy)" : "date",
    "gwl(cm)" : "gwl_cm",
    "coor_y(dd)" : "lat",
    "coor_x(dd)" : "lon",
}

# Read pkeg's data
pkeg_df = pd.read_csv(pkeg_path, sep=";")

# Rename columns
pkeg_df.rename(columns=pkeg_cols_rename, inplace=True)

# combine "kode_perusahaan"	and "kode_titik" to create a unique id
pkeg_df.loc[:, "id"] = pkeg_df["kode_perusahaan"] + "_" + pkeg_df["kode_titik"]

# Convert date column to datetime
pkeg_df.loc[:, "date"] = pd.to_datetime(pkeg_df["date"], dayfirst=True)

# set a new column for source
pkeg_df["source"] = "pkeg"

# Only select columns that are needed
pkeg_df = pkeg_df[DATA_COLS]
pkeg_df.head()

### Previous BRG data

In [ ]:
old_brg_df = pd.read_csv(old_brg_path)
old_brg_df.loc[:, "source"] = "old_brg"
old_brg_df.loc[:, "date"] = pd.to_datetime(old_brg_df["date"], dayfirst=True)

# Multiply gwl_cm by 100 to convert it to cm
old_brg_df["gwl_cm"] = old_brg_df["gwl_cm"] * 10
old_brg_df = old_brg_df[DATA_COLS]
old_brg_df.head()

### Concatenate all data sources

In [ ]:
# Merge the two dataframes
df = pd.concat([brg_df, pkeg_df, old_brg_df], ignore_index=True)

# save the dataframe as csv
df.to_csv("data/field_data_all_with_old.csv", index=False)

In [ ]:
# Remove duplicate dates for each id using the mean value
# Group by 'id' and 'date' and calculate the mean value for 'gwl_cm' while keeping other columns

print("Before removing duplicates", len(df))

agg_dict = {'source':'first','lon':'first','lat':'first','gwl_cm':'mean'}
df = df.groupby(['id','date']).agg(agg_dict).reset_index()

print("After removing duplicates", len(df))

In [ ]:
# Below I will remove the duplicated coordinates IDS and keep the first one
# get unique lon-lat pairs
unique = df[["id", "lon", "lat"]].drop_duplicates()

# Get duplicated lon-lat pairs
duplicated = unique[unique.duplicated(subset=["lon", "lat"], keep=False)]

duplicated = duplicated.drop_duplicates(subset=["lon", "lat"], keep="first")

# Get the duplicated ids
duplicated_ids = duplicated["id"].unique()

# # get dataframe without duplicated ids

df = df[~df["id"].isin(duplicated_ids)]

df.to_csv("data/field_data_unique_coords.csv", index=False)
df

In [ ]:
# get unique coordinates for each station
stations = df[["id", "source", "lon", "lat"]].drop_duplicates()


# Convert to GeoDataFrame
from geopandas import GeoDataFrame
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(stations.lon, stations.lat)]
stations_gdf = GeoDataFrame(stations, geometry=geometry)
stations_gdf.crs = "EPSG:4326"
stations_gdf.to_file("data/0_shp/unique_stations_no_repeated.shp")